Pregunta1: Qué día hubo la mayor cantidad de personas en cuarentena en cada región?

Pregunta2: Qué día hubo la mayor cantidad de personas en cuarentena en el país?

In [116]:
import pandas as pd
import numpy as np 
import matplotlib as npl 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
plt.style.use('default')

In [117]:
df = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto29/Cuarentenas-Totales.csv?raw=true', sep=',')

In [118]:
df2 = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto1/Covid-19.csv?raw=true', sep=',')

In [119]:
df.head()

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m,Region,n_REGION
0,1,Arica,Histórica,Área Urbana Completa,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,28050418.80,48722.30001,15,R15
1,2,Santiago Norte,Histórica,Sector Específico,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,Corresponde a la extensión de la cuarentena en...,19939218.57,21599.35134,13,R13
2,3,Puente Alto Poniente,Histórica,Sector Específico,2020-04-09 22:00:00,2020-05-08 22:00:00,13201,Corresponde al sector delimitado por la Av. Co...,42364268.21,30428.05152,13,R13
3,4,El Bosque,Histórica,Comuna completa,2020-04-16 22:00:00,2020-09-21 05:00:00,13105,Aplicada a la totalidad de la comuna. Se incor...,20622713.23,18788.87075,13,R13
4,5,San Bernardo Nororiente,Histórica,Sector Específico,2020-04-16 22:00:00,2020-05-08 22:00:00,13401,Corresponde al sector delimitado por los limit...,16326216.83,24929.14572,13,R13


In [120]:
df2.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2021-09-10,2021-09-13,2021-09-17,2021-09-20,2021-09-24,2021-09-27,2021-10-01,2021-10-04,2021-10-08,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,29763.0,29836.0,29929.0,30002.0,30128.0,30239.0,30376.0,30463.0,30536.0,12335.2
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,67.0,67.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,205.0,207.0,208.0,207.0,211.0,215.0,213.0,213.0,213.0,8469.2
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,158.0,163.0,161.0,163.0,165.0,165.0,167.0,170.0,175.0,NaN


In [121]:
limpio_df = df.drop(['Superficie en m2', 'Perímetro en m', 'Detalle'], axis = 1)

In [122]:
columnas_df2 = df2[['Region','Codigo region','Comuna','Codigo comuna','Poblacion']]

In [123]:
nul_df2= df2.dropna()

In [124]:
nuevo_nombre = limpio_df.rename(columns={'Código CUT Comuna': 'Codigo comuna'})

In [125]:
mix = nuevo_nombre.merge(nul_df2, on= 'Codigo comuna')

In [126]:
limpio_mix = mix.drop(['ID', 'Nombre', 'Estado', 'Alcance'], axis = 1)

In [127]:
limpio_mix["Fecha de Inicio"]=limpio_mix["Fecha de Inicio"].apply(lambda x:x.split(" ")[0])
limpio_mix["Fecha de Término"]=limpio_mix["Fecha de Término"].apply(lambda x: x.split(" ")[0])

In [128]:
limpio_mix["Fecha de Inicio"]=pd.to_datetime(limpio_mix["Fecha de Inicio"])
limpio_mix["Fecha de Término"]=pd.to_datetime(limpio_mix["Fecha de Término"])

In [129]:
limpio_mix['Total días en cuarentena'] = limpio_mix['Fecha de Término'].sub(limpio_mix['Fecha de Inicio'], axis=0)

In [130]:
limpio_mix['Total días en cuarentena'] = limpio_mix['Total días en cuarentena'].dt.days.astype('int16')

In [131]:
comunas = []
for i in limpio_mix['Comuna']:
    comunas.append(i)

cuarentena = []
for i in limpio_mix['Total días en cuarentena']:
    cuarentena.append(i)


In [132]:
dict_lista = dict(zip(comunas, cuarentena))
reps = [dict_lista[val] if val in dict_lista else 1 for val in limpio_mix['Comuna']]

In [133]:
datos_cuarentenas = limpio_mix.loc[np.repeat(limpio_mix.index.values, reps)]

In [134]:
region1 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==1)
region2 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==2)
region3 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==3)
region4 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==4)
region5 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==5)
region6 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==6)
region7 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==7)
region8 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==8)
region9 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==9)
region10 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==10)
region11 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==11)
region12 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==12)
region13 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==13)
region14 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==14)
region15 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==15)
region16 = datos_cuarentenas.where(datos_cuarentenas['Codigo region']==16)

In [135]:
region1 = region1.dropna()
region2 = region2.dropna()
region3 = region3.dropna()
region4 = region4.dropna()
region5 = region5.dropna()
region6 = region6.dropna()
region7 = region7.dropna()
region8 = region8.dropna()
region9 = region9.dropna()
region10 = region10.dropna()
region11 = region11.dropna()
region12 = region12.dropna()
region13 = region13.dropna()
region14 = region14.dropna()
region15 = region15.dropna()
region16 = region15.dropna()